### 文本分类词嵌入（Word2Vec）、LSTM、Attention机制

- 学习目标：掌握使用词嵌入（Word2Vec）、LSTM 和 Attention 机制进行文本分类的技术，并在 IMDB 电影评论情感分析任务中实现准确率 > 85%。
- 学习内容：
  - 理解词嵌入（Word2Vec）的基本原理和实现方法。
  - 学习 LSTM 网络结构及其在文本分类中的应用。
  - 掌握 Attention 机制的原理，并将其应用于文本分类任务。

#### 词嵌入（Word2Vec）

词嵌入是一种将词语转换为向量表示的方法，使得相似的词语在向量空间中距离较近。Word2Vec 是一种常用的词嵌入方法，它通过训练神经网络来学习词语的向量表示。

In [ ]:
# 示例代码：训练 Word2Vec 模型
from gensim.models import Word2Vec

# 假设 sentences 是一个包含文本数据的列表，每个元素是一个句子（词语列表）
sentences = [['this', 'is', 'a', 'sample', 'sentence'], ['another', 'example', 'sentence']]

# 训练 Word2Vec 模型
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# 获取词语 'sample' 的向量表示
vector = model.wv['sample']
print(vector)


#### LSTM 网络结构

LSTM（Long Short-Term Memory）是一种特殊的 RNN（Recurrent Neural Network），它通过引入门控机制来解决传统 RNN 中的梯度消失和梯度爆炸问题。LSTM 在处理序列数据（如文本）时表现出色。

In [ ]:
# 示例代码：构建 LSTM 模型
import torch
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout):
        super(LSTMModel, self).__init__()
        # 词嵌入层
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        # LSTM 层
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, bidirectional=bidirectional, dropout=dropout, batch_first=True)
        # 全连接层
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        # Dropout 层
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        # 嵌入输入
        embedded = self.dropout(self.embedding(x))
        # LSTM 输出
        lstm_out, _ = self.lstm(embedded)
        # 全连接层输出
        output = self.fc(lstm_out[:, -1, :])
        return output


#### Attention 机制

Attention 机制是一种增强模型对重要信息关注能力的方法。它通过为输入序列中的每个元素分配一个权重，从而使模型能够更好地捕捉全局信息。

In [ ]:
# 示例代码：在 LSTM 模型中加入 Attention 机制
class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super(Attention, self).__init__()
        self.attention = nn.Linear(hidden_dim, 1)
    
    def forward(self, lstm_out):
        attn_weights = torch.softmax(self.attention(lstm_out), dim=1)
        context = torch.sum(attn_weights * lstm_out, dim=1)
        return context

class LSTMModelWithAttention(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout):
        super(LSTMModelWithAttention, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, bidirectional=bidirectional, dropout=dropout, batch_first=True)
        self.attention = Attention(hidden_dim * 2 if bidirectional else hidden_dim)
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        embedded = self.dropout(self.embedding(x))
        lstm_out, _ = self.lstm(embedded)
        context = self.attention(lstm_out)
        output = self.fc(context)
        return output
'''
传统 LSTM 和加入 Attention 机制的 LSTM 的区别：
传统 LSTM 模型直接将 LSTM 层的最后一个时间步的输出传递给全连接层。
加入 Attention 机制的 LSTM 模型通过计算每个时间步的注意力权重，得到上下文向量，再将上下文向量传递给全连接层。
'''


In [ ]:
# 训练和评估模型
# ...existing code...
